- AI 에이전트

  - 사용자의 목표를 달성하기 위해 자율적으로 환경을 인지하고, 데이터를 처리하며, 의사결정을 내려 실제 행동을 수행하는 인공지능 시스템
  - 특징 : 자율성, 인지, 추론,계획, 행동, 협력
    - 챗봇 : 주어진 질문에 답변하는 역할
    - AI 에이전트 : 업무 실행, 프로세스 자동화, 의사결정 까지 수행
- GPT tools

  - 여러 tool 을 정의하면 GPT 는 상황에 맞게 선택하거나 동시에 호출해서 문제를 해결합니다. AI 에이전트 구현에 사용.
  - tools : (function_call은 단일 함수만 호출 합니다.)
    - gpt-4 모델 부터 도입.
    - 여러 함수 또는 API 등등을 동시에 호출.
    - 리턴값 중 tool_calls 항목은 배열(리스트)

In [1]:
!python ai_responseV3.py

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
from ai_responseV3 import get_ai_response_tools, get_first_response_tools, get_followup_response_tools
load_dotenv()

client = OpenAI()

In [3]:
tools = [
    {
    "type": "function",
    "function": {
                "name": "get_current_time_tz",
                "description": "현재 시간 출력 HH:MM:SS format",
                # 함수의 인자를 정의
                "parameters": {
                    "type": "object",
                    "properties": {
                        "timezone": {
                            "type": "string",
                            "description": "Time zone in 'Area/Location' format, e.g., 'Asia/Seoul', 'America/New_York'. Default is 'Asia/Seoul'."
                        }
                    },
                    "required": ['timezone']
                }
            },
    },
    { 
        "type": "function",
        "function": {
        "name": "get_current_date_tz",
        "description": "현재 날짜 출력 YYYY 년 MM 월 DD 일 format",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "Time zone in 'Area/Location' format, e.g., 'Asia/Seoul', 'America/New_York'. Default is 'Asia/Seoul'."
                }
            },
            "required": ['timezone']
        }}
    }
]

In [4]:
def get_first_response_tools(question):
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. using locale language."},
            {"role": "user", "content": question}
        ],
        tools=tools,
        tool_choice="auto"
    )
    return response

In [5]:
question = '서울과 뉴욕의 시차를 알려줘'
# question = '지금 뉴욕의 날짜와 시간을 알려줘'   
# question = '서울과 뉴욕의 날짜와 시간을 알려줘'  
response = get_first_response_tools(question)
print(type(response))
tool_calls = response.choices[0].message.model_dump_json(indent=2)
print(tool_calls)

<class 'openai.types.chat.chat_completion.ChatCompletion'>
{
  "content": null,
  "refusal": null,
  "role": "assistant",
  "annotations": [],
  "audio": null,
  "function_call": null,
  "tool_calls": [
    {
      "id": "call_NDmFj30kZHsJ6Xx7Xtn0nn93",
      "function": {
        "arguments": "{\"timezone\": \"Asia/Seoul\"}",
        "name": "get_current_time_tz"
      },
      "type": "function"
    },
    {
      "id": "call_LgHk12JSVqOt1PdVol685YGO",
      "function": {
        "arguments": "{\"timezone\": \"America/New_York\"}",
        "name": "get_current_time_tz"
      },
      "type": "function"
    }
  ]
}


In [6]:
"""
"tool_calls": [
{
    "id": "call_69WQgFuuzBmArYsglQ9TQ0Dr",
    "function": {
            "arguments": "{\"timezone\": \"America/New_York\"}",
            "name": "get_current_date_tz"
            },
    "type": "function"
},
{
    "id": "call_1dZ2ccj1kgW6QOs5xMz1pOB6",
    "function": {
            "arguments": "{\"timezone\": \"America/New_York\"}",
            "name": "get_current_time_tz"
            },
    "type": "function"
}
]
"""
import json
from ai_responseV2 import get_current_date_tz, get_current_time_tz
tool_results = []
for tool in response.choices[0].message.tool_calls:
    fn_name = tool.function.name
    args = json.loads(tool.function.arguments)
    
    # 실제 Python 함수 실행
    if fn_name in globals():
        result = globals()[fn_name](**args)
        tool_results.append({"name": fn_name, "result": result})
    else:
        tool_results.append({"name": fn_name, "result": f"Unknown function: {fn_name}"})

log : 현재 시간 : 15:38:00 Asia/Seoul
log : 현재 시간 : 01:38:00 America/New_York


In [7]:
tool_results

[{'name': 'get_current_time_tz', 'result': '15:38:00'},
 {'name': 'get_current_time_tz', 'result': '01:38:00'}]

In [8]:
def get_followup_response_tools(tool_results):
    result_text = "\n".join([f"{t['name']} 결과는 {t['result']} 입니다." for t in tool_results])
    followup_response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. using locale language."},
            {"role": "user", "content": f"{question} 에 대해 다음 결과를 이용해 사용자가 이해하기 쉽게 자연스러운 최종 답변을 만들어줘:\n{result_text}"}
        ],
        tools=tools
    )
    return followup_response

In [9]:
final_response = get_followup_response_tools(tool_results)

In [10]:
print(final_response.model_dump_json(indent=2))
print(f'💬AI : {final_response.choices[0].message.content}')

{
  "id": "chatcmpl-CbhjDGPFZo6U9kt7Q9k6ZFNbcj1K6",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "서울과 뉴욕의 현재 시각을 비교해보면, 서울은 15시 38분이고 뉴욕은 01시 38분입니다. 즉, 서울이 뉴욕보다 14시간 앞서 있습니다. 따라서 서울과 뉴욕의 시차는 14시간입니다.",
        "refusal": null,
        "role": "assistant",
        "annotations": [],
        "audio": null,
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1763102263,
  "model": "gpt-4.1-mini-2025-04-14",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_4c2851f862",
  "usage": {
    "completion_tokens": 63,
    "prompt_tokens": 219,
    "total_tokens": 282,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
💬AI : 

In [11]:
question = '서울과 뉴욕의 시차알려줘'  #'서울은 어느 나라야?'
final_response = get_ai_response_tools(question)
print(f'💬AI : {final_response}')

log : 현재 시간 : 15:38:03 Asia/Seoul
log : 현재 시간 : 01:38:03 America/New_York
💬AI : 현재 서울 시간은 15시 38분 03초이고, 뉴욕 시간은 01시 38분 03초입니다. 따라서 서울과 뉴욕의 시차는 14시간입니다. 서울이 뉴욕보다 14시간 빠릅니다.
